In [27]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey, X25519PublicKey
from cryptography.hazmat.primitives.serialization import load_der_public_key, load_der_private_key
from cryptography.hazmat.primitives.serialization import Encoding
from cryptography.hazmat.primitives.serialization import PublicFormat
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding

### Pomožne funkcije za nalaganje vsebin z diska

Medtem ko so bili simetrični ključi preprosto naključne vrednosti imajo javni in zasebni ključi bolj dovršeno strukturo.

Za postroj in zapis javnih in zasebnih ključev imamo standarde, denimo X.509, ki določa različne načine zapisa ključev, certifikatov in sorodnih reči. 

S tem se tokrat ne bomo ukvarjali. Za lažje branje podatkov z diska le uporabite spodnje funkcije.

In [47]:
def load_bytes(filename):
    with open(filename, "rb") as h:
        return h.read() 
    
def load_public_key(filename):
    """Prebere javni ključ z diska"""
    return load_der_public_key(load_bytes(filename))

def load_secret_key(filename):
    """Prebere zasebni ključ z diska"""
    return load_der_private_key(load_bytes(filename), None)

## Naloga 1: Digitalni podpis RSA-PSS

Implementirajte shemo digitalnega podpisa RSA, tako da podate implementacijo algoritmov za ustvarjanje ključev, podpisovanja in preverjanja podpisa. Pri tem si pomagajte z dokumentacijo knjižnice cryptography: [RSA @ cryptography](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/)

Specifikacije so naslednje:

Algoritem `gen(int)`
- opcijsko vzame velikost ključa (modula); privzeta vrednost naj bo 2048 bitov;
- eksponent e naj bo 65537
- vrne naj par `(pk, sk)`

In [39]:
def gen_rsa(key_size=2048):
    sk = rsa.generate_private_key(public_exponent=65537, key_size=key_size)
    return sk.public_key(), sk

gen_rsa()

(<cryptography.hazmat.backends.openssl.rsa._RSAPublicKey at 0x7f92ce3a3c70>,
 <cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey at 0x7f92ec187f10>)

In [38]:
def test_gen_rsa():
    from cryptography.hazmat.backends.openssl.rsa import _RSAPublicKey, _RSAPrivateKey
    pk, sk = gen_rsa()
    assert pk.key_size == 2048
    assert type(pk) == _RSAPublicKey
    assert type(sk) == _RSAPrivateKey
    
    pk, sk = gen_rsa(4096)
    assert pk.key_size == 4096
    
test_gen_rsa()

Podpisni algoritem vzame dva argumenta:
- `sk` zasebni ključ kot ga vrne algoritem `gen_rsa()`,
- `ct` čistopis v bajtih

in vrne bajte, ki predstavljajo vrednost digitalnega podpisa. 

Implementirajte različico RSA-PSS. Za MGF uporabite `MGF1`, zgoščevalna funkcija naj bo v obeh primerih `SHA256`, dolžino soli ročno nastavite na `32`.

In [40]:
def sign_rsa(sk, message):
    return sk.sign(
        message,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=32),
        hashes.SHA256()
    )

sign_rsa(gen_rsa()[1], b"test")

b"\xb9J\x1b.\xaaQ\x8c?\xb4\xb1R\xbcR~E.qy\xcel/\xf2\xd2\x1d\xda\xf88$0Fz7\xb6\xc6\xe4/A<5\x7f`&\xba\x9a\x0e\x19\xf0O\x15\x9c1\x96\xc61\xa1\xcd}!;\x05]\xd8\xd93\t\xf2\xf0\xf7 X\x86\x90KW\x07_n1\x98M\xdc\x9b\xc2\x18\r\xd2%\\\xd2\x99\x08q\xdd\xb78\x19z\x12\xd9\x9c\x94\x08\xf8Gw\xf1R\x8f\xea\xfa/0\x18V\xc6\xb8s'\xe8\x98qv\xa4\xc4[N\xe9\x07r\xabnd\xf8\x7f\x03\x822\x0bd3)\x95\xf3\xb1#\x01\xa9\xb2\xbf\xc7K\xd3m\xf7P\xbb\x92w9\x15\xf4\xa1I\x9d\xbb\x12\xe4><i\xfc\xd9\x98\x8dJ\xdc\x8a?\x01\xb6)\xe6\xcb\xd2H\xed\xde[{\x0e\xc5=\x03\x90\xd0\xb5\x1c\xc9\x14W\xeajF\x04l3\xdd4\xedqy\x88\xd5-\xfc\xc4\xd4\x06\x8b>\x84!\x04\xff\xe2t\xe1\x80\x00\xd44Y\x112\xf0n\xdb\n+\xcf\xeb\xbf\x17\x12\xf8\x1b\x918\x17\xf2&\\6hL-"

Algoritem preverjanja vzame tri argumente:
- `pk` javni ključ kot ga vrne algoritem gen_rsa(),
- `pt` čistoips v bajtih,
- `sig`, pridruženi podatki v bajtih.

Kot pri podpisovanju za MGF uporabite `MGF1`, zgoščevalna funkcija naj bo v obeh primerih `SHA256`, dolžino soli ročno nastavite na `32`.

Kot rezultat vrnite `True` natanko takrat ko preverjanje podpisa uspe. Sicer vrnite `False`.

In [62]:
def verify_rsa(pk, message, sig):
    try:
        pk.verify(sig, message, 
            padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=32),
            hashes.SHA256())
        return True
    except:
        return False
    
_rsa_key = gen_rsa()
verify_rsa(_rsa_key[0], b"test", sign_rsa(_rsa_key[1], b"test"))

True

In [25]:
def test_sign_verify_rsa():
    pk, sk = gen_rsa()
    
    assert verify_rsa(pk, b"test", sign_rsa(sk, b"test"))
    assert not verify_rsa(pk, b"test1", sign_rsa(sk, b"test"))    

test_sign_verify_rsa()

## Naloga 2: Implementacija RSA-PSS (Java)

V Javanskem projektu implementirajte RSA-PSS v datoteki `RSAPSSSignature.java`. 

Začnite z ogledom datotek `RSASignatureExample` ter  `Ed25519SignatureExample.java`, ki sta primer podpisa in njegovega preverjanja za algoritma RSA-PKCS1 ter Ed25519.

Primer s RSA-PSS je zelo podoben, le da boste kot algoritem za ustvarjanje ključa nastavili na `RSA-PSS` ter podobno še za podpisni algoritem ter algoritem za preverjanje.

Pri tem pazite na sledeče zahteve gelde RSA-PSS:
- za MGF uporabite MGF1,
- zgoščevalna funkcija naj bo v obeh primerih SHA256 in
- dolžino soli ročno nastavite na 32.

To storite z uporabo razreda `PSSParameterSpec` kot podaja sledeč primer.

```java
Signature signer = // ....
signer.setParameter(new PSSParameterSpec("SHA-256", "MGF1", MGF1ParameterSpec.SHA256, 32, 1));
signer.initSign(key);
```

Implementirajte funkcije `gen()`, `sign()` in `verify()`.

### Preverjanje: RSA-PSS (Python)

Ko bo Javanska implementacija zaključena, se bodo po uspešno izvedenem programu ustvarile  datoteke `rsa.pk`, `rsa.sk`, `rsa.msg` ter `rsa.sig`, ki vsebujejo javni in zasebni ključ ter sporočilo in podpis.

Datoteke naložite v Python in z uporabo funkcije `verify_rsa(pk, message, sig)` preverite veljavnost podpisa še v pythonu.

In [72]:
assert verify_rsa(
    load_public_key("rsa.pk"), # javni kljč
    load_bytes("rsa.msg"),     # sporočilo
    load_bytes("rsa.sig")      # podpis
)

## Naloga 3: Digitalni podpis Ed25519 (Java)

Zelo podobna naloga kot prejšnja, le da uporabimo drugo podpisno shemo, tokrat Ed25519. Gre za eno novejših podpisnih shem, ki se pogosto uporablja in temelji na eliptični krivulji 25519.

Naloga se nahaja v datoteki `Ed25519Signature.java`. Implementirajte funkcije `gen()`, `sign()` in `verify()`. 

Za algoritme za ustvarjanje ključev, podpisovanja in preverjanja izberite ustrezne nastavitve: le poiščite pravilna [imena algoritmov v dokumentaciji.](https://docs.oracle.com/en/java/javase/21/docs/specs/security/standard-names.html#cipher-algorithms)

## Naloga 4: Digitalni podpis Ed25519 (Python)

Ko bo Javanska implementacija uspešno zaključena, se bodo po uspešno izvedenem programu ustvarile  datoteke `ed25519.pk`, `ed25519.sk`, `ed25519.msg` ter `ed25519.sig`, ki vsebujejo javni in zasebni ključ ter sporočilo in podpis.

Datoteke naložite v Python in preverite, [ali je podpis veljaven.](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ed25519/#signing-verification)

In [70]:
def verify_ed25519(pk, message, sig):
    try:
        pk.verify(sig, message)
        return True
    except:
        return False

assert verify_ed25519(
    load_public_key("ed25519.pk"), # javni ključ
    load_bytes("ed25519.msg"),     # sporočilo
    load_bytes("ed25519.sig")      # podpis
)

## Naloga 5: Moderni El Gamal (ECIES) (Java)

Za konec implementirajmo še hibridno šifro El Gamal (znano pod modernim imenom ECIES) v Javi, delno pa tudi v Pythonu.

Implementacija naj sledi algoritmu `gen()`, `enc(pk, pt)` in `dec(sk, ct)`, ki smo jih podali na predavanjih. Upoštevajte pa sledeče:

- Pri ustvarjanju ključev uporabite algoritem `X25519`.
- Tekom šifriranja (in dešifriranja) bo potrebno izvesti protokol Diffie--Hellman: v Javi izberite algoritem `XDH`.
- Za zgoščevalno funkcijo uporabite `SHA-256`.
- Simetrično overjeno šifriranje izvedite s `ChaCha20-Poly1305`.
- Celoten tajnopis naj sestoji iz vrednosti `B || IV || CT`, kjer `B` predstavlja (začasni) javni ključ ustvarjen pri šifriranju, `IV` naključno vrednost uporabljeno pri simetrični šifri ter `CT` simetrični tajnopis dobljen s šifro `ChaCha20-Poly1305`.

Prepričajte se, da implementacija v javi deluje pravilno. Tekom izvedbe, bi morala ustvariti datoteke, ki predstavljajo javni in zasebni ključ ter tajnopis. 

## Naloga 6: Moderni El Gamal (ECIES) (Python)

Na koncu implementirajte še dešifrirni algoritem v Pythonu: spodnja koda naloži zasebni ključ ter tajnopis in ju skuša dešifrirati.

Pri implementaciji v Pythonu bodite pozorni:
- Tajnopis razčlenite v 3 dele:
  - Prvih 44 bajtov predstavlja začasni javni ključ `B`
  - Naslednjih 12 bajtov predstavlja IV
  - Preostali bajti predstavljajo simetričen tajnopis (in značko)
- Preden lahko začasni javni ključ `B` uporabite, ga morate pretvoriti iz bajtov v ustrezen objekt s funkcijo `load_der_public_key()`.
- A kot ta isti začasni javni ključ `B` uporabite v zgoščevalni funkciji pri izpeljavi simetričnega ključa, morate uporabiti prvotno 44-bajtno različico.

In [76]:
def decrypt_ecies(sk, ct):
    B = ct[:44]
    iv = ct[44:56]
    c = ct[56:]
    
    dhs = sk.exchange(load_der_public_key(B))
    h = hashes.Hash(hashes.SHA256())
    h.update(B)
    h.update(dhs)
    key = h.finalize()
    chacha = ChaCha20Poly1305(key)
    return chacha.decrypt(iv, c, None)

pt = decrypt_ecies(
    load_secret_key("ecies.sk"), # skrivni ključ
    load_bytes("ecies.ct") # tajnopis
)
print("Dešifrirano:", pt.decode("utf8"))
assert pt == load_bytes("ecies.msg")

Dešifrirano: A test message.
